In [1]:
#import modules and set up environment 
import os
import sys
path = "../../src/"

sys.path.append(path)
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import jax.numpy as jnp
from jax import jit, grad, vmap, hessian, scipy, random

#import sgmcmc code 
import models.linear_regression.linear_regression as lr
from samplers import sgd as sgd
from samplers import sgld as sgld
from samplers import sgldps as sgldps
from samplers import sgldcv as sgldcv
from samplers import sgldcvps as sgldcvps

key = random.PRNGKey(11)
from metrics import imq_KSD

/opt/anaconda/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
#set up model-specific gradient functions
gradf_0 = lr.gradf_0
gradf_i_batch = lr.gradf_i_batch
post_var = lr.post_var

In [3]:
#load in the data
file_path = "../../data/real/casp_scaled.csv"
data = pd.read_csv(file_path)
dat_array = data.values[:]
N = dat_array.shape[0]
x = np.column_stack([np.ones(N), dat_array[:, 1:]])
y = dat_array[:,0]

#set up model parameters
dim = x.shape[1] 

### Importing csv files

In [4]:
methods = ["sgldcv", "sgldcvps", "asgldcv", "asgldcvps"]
samples_csv = dict()
#iterations
Niter = np.int64(100/0.01)

for i in range(len(methods)):
    method = methods[i]
    items = glob.glob(f"./out/lr_{method}_samples.csv")[0]
    samples_csv[method] = pd.read_csv(items).iloc[(Niter+1):].reset_index(drop=True)

In [5]:
grads_csv = dict()

#gradients for all methods
for i in range(len(methods)):
    method = methods[i]
    items = glob.glob(f"./out/lr_{method}_grads.csv")[0]
    grads_csv[method] = pd.read_csv(items).iloc[:].reset_index(drop=True)

### KSD evaluation

In [6]:
Nrep = 10

In [7]:
for k in tqdm(range(len(methods)), desc= "Methods"):
    method = methods[k]
    idx=np.arange(Niter, 0, -200)[::-1]-1
    ksd_arr = np.zeros((idx.shape[0], Nrep))
    for j in range(Nrep):
        for i in range(idx.shape[0]):
            index = idx[i]
            samples = samples_csv[method].values[:index, (dim*(j+1)):dim*(j+2)]
            grads = grads_csv[method].values[:index, (dim*(j+1)):dim*(j+2)]
            ksd_arr[i, j] = np.log10(imq_KSD(samples, grads))
        
    ksd_arr_df = pd.DataFrame(ksd_arr)
    ksd_arr_df.to_csv(f"./out/ksdcasp_{method}.csv", index=False)

Methods: 100%|██████████| 4/4 [46:41<00:00, 700.39s/it]
